# Outline

- Simple hashing - hotel with 26 rooms, labeled A-Z. Guests are assigned to room based on their first name initial. If room occupied guest is not admitted.
- Not a good business model. Lots of demand for room `J` and `M` because John, Mary very popular names. Less demand for `X` or `Z`.
- Superfast to tell if a person is a guest at the hotel. If room corresponding to person's first name initial is not empty, person with that present. $\mathcal O(1)$.
- Brief/simple intro to $\mathcal O$ notation.
- Many room remain empty. Introduce probing. John goes to `J`. James cannot go to `J` because John is there but what if the next room `K` is available. James can go there. Now, it may take 2 steps to find if James is in hotel. We expect him in `J` but we'll also probe `K`. Still faster than linear search or binary search.
- Discuss probing.
- Probing requires that we take over rooms that should be available for other guests.
- How about moving roll-in beds to a room as more guest with the same name initial arrive at the hotel (ok, it will get a bit crowded, but we are not a 5-star hotel).
- Introduce simple module based hashing. Instead of a hotel with 26 rooms, for A, B, .., Z, use `ord(initial)%capacity` to determine room number.
- In building a class (Hotel Alphabetical) discuss some standardized behavior (based on software contracts). For example, the objects must have an `exists -> bool` function. A `remove -> guest` etc.

# Summary of assignments

This assignment comprises \*\*\* problems.

- Write the `exists` method
- Write the `delete` method


In [ ]:
class Guest:

    """Class representing guests for our Hotel Alphabetical. These objects
    are essentially nodes in a linked list. Each hotel room is effectively a
    linked list of guests, placed in that room based on some hashing function.
    """

    def __init__(self, name: str):
        self.name = name
        self.next = None  # Pointer to the next guest in the linked list

    def __repr__(self):
        return f"Guest({self.name})"

    def set_next(self, next_guest):
        self.next = next_guest

    def get_next(self):
        return self.next

    def has_next(self):
        return self.next is not None

    def get_name(self):
        return self.name

In [ ]:
class HotelAlphabetical:

    """Class representing a hotel where guests are stored in rooms based on
    the first initial of their names. Each room is a linked list of guests.
    """

    _DEFAULT_CAPACITY = 26
    _ASCII_LEFT_EDGE = ord('A')
    _ASCII_RIGHT_EDGE = ord('Z')
    _EMPTY = "boohoo, your hotel is empty."

    def __init__(self, capacity: int = _DEFAULT_CAPACITY):
        self._capacity = capacity
        self._hotel = [None] * capacity  # Array of linked lists for each letter
        self._size = 0
    
    def _get_index(self, name: str) -> int:
        """ Compute the index in the hotel array based on the first 
        initial of the guest's name. """
        # Default to 0 if name is None or empty or not A-Z
        room_index = 0
        if name is not None and len(name)>0:
            # DISCUSSION POINT: should we be computing the first initial
            # here or should it be done in object Guest?
            initial_ascii = ord(name.upper()[0])
            if self._ASCII_LEFT_EDGE <= initial_ascii <= self._ASCII_RIGHT_EDGE:
                room_index = initial_ascii % self._capacity
        return room_index
    
    def add_guest(self, name:str) -> None:
        """ Add a guest to the hotel."""
        # Compute the room index based on the first initial of the name
        room = self._get_index(name)
        # Create a new guest object
        guest = Guest(name)
        # Insert the guest at the front of the linked list for that room
        if self._hotel[room] is None:
           self._hotel[room] = guest
        else:
            guest.set_next(self._hotel[room])
            self._hotel[room] = guest
        # Increment the current occupancy of the hotel
        self._size += 1
    
    def exists(self, name) -> bool:
        """ Check if a guest with the given name exists in the hotel."""
        found = False
        # Determine the expected room based on the first initial of the name
        expected_room = self._get_index(name)
        # Traverse the linked list in that room to find the guest
        guest_in_room = self._hotel[expected_room]
        while guest_in_room is not None and not found:
            found = guest_in_room.get_name() == name
            guest_in_room = guest_in_room.get_next()
        return found

    def __repr__(self) -> str:
        hotel_string = self._EMPTY
        if self._size > 0:
            hotel_string = f"\nThere are {self._size} guest(s) in your hotel."
            for room in range(self._capacity):
                if self._hotel[room] is not None:
                    hotel_string += f"\nRoom {room:02d}: "
                    guest_in_room = self._hotel[room]
                    while guest_in_room is not None:
                        hotel_string += f"{guest_in_room.get_name()} --> "
                        guest_in_room = guest_in_room.get_next()
                    hotel_string += "None"
        return hotel_string


In [ ]:
test = HotelAlphabetical(4)
# Obligatory statement about how bad idea is to write multi-statement lines
# but it is just for testing purposes here
test.add_guest("Alice"); test.add_guest("Aaron"); test.add_guest("Ava")
test.add_guest("Bob"); test.add_guest("Charlie"); test.add_guest("David")
test.add_guest("Eve"); test.add_guest("Zara"); test.add_guest("Yvonne")
test.add_guest("Xander"); test.add_guest("Mallory"); test.add_guest("Trent")
test.add_guest("Peggy"); test.add_guest("Victor"); test.add_guest("Walter") 
print(test)



There are 15 guest(s) in your hotel.
Room 00: Peggy --> Trent --> Xander --> David --> None
Room 01: Mallory --> Yvonne --> Eve --> Ava --> Aaron --> Alice --> None
Room 02: Victor --> Zara --> Bob --> None
Room 03: Walter --> Charlie --> None
